## Feature Engineering

The purpose of this notebook is to create features that will allow the model to better identify if a transaction is fraudulent above and beyond the given features in the dataset.

I will create the following features:
1. Whether a transaction was completed in the customer's local currency or not
3. Whether a transaction was completed in neither the customer nor the counterparty country local currency. This behavior seems suspect to me, so I want to put it in the model.
4. Whether the transaction was completed outside normal hours. "Outside normal" hours here I am assuming between midnight and 6 am local times.
5. The counteryparty's country of origin. Assumption here that companies in some countries could be more likely to exhibit fraud than others.
6. Lastly, converting catgorical variables into dummy variables.

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
trxns = pd.read_csv('../data/preprocessed_trxns.csv')
trxns['converted_time'] = pd.to_datetime(trxns.converted_time)
trxns.head()

,customer,customer_country,fraud_flag,timestamp,counterparty,counterparty_country,type,ccy,amount,hr_rounded,converted_time,converted_amount,scaled_amount
0,C12976926337644,UK,N,2021-03-20 05:59:25,77697395249488,CN,OTHER,JPY,48625.25,6,2021-03-19 21:59:25,330.6517,-0.186732
1,C12976926337644,UK,N,2021-05-02 03:00:38,77697395249488,CN,PAYMENT,GBP,20637.21,3,2021-05-01 19:00:38,23526.4194,-0.115031
2,C12976926337644,UK,N,2021-05-23 22:15:15,77697395249488,CN,PAYMENT,USD,56691.27,22,2021-05-23 14:15:15,56691.2700,-0.012516
3,C12976926337644,UK,N,2021-07-07 21:36:50,77697395249488,CN,PAYMENT,CNY,343864.71,22,2021-07-07 13:36:50,48141.0594,-0.038945
4,C12976926337644,UK,N,2021-09-02 00:14:03,77697395249488,CN,PAYMENT,USD,57247.35,0,2021-09-01 16:14:03,57247.3500,-0.010797


In [3]:
trxns['outside_normal_hrs'] = np.where((trxns.converted_time.dt.hour > 0) & (trxns.converted_time.dt.hour < 7), 1, 0)
trxns.head()

,customer,customer_country,fraud_flag,timestamp,counterparty,counterparty_country,type,ccy,amount,hr_rounded,converted_time,converted_amount,scaled_amount,outside_normal_hrs
0,C12976926337644,UK,N,2021-03-20 05:59:25,77697395249488,CN,OTHER,JPY,48625.25,6,2021-03-19 21:59:25,330.6517,-0.186732,0
1,C12976926337644,UK,N,2021-05-02 03:00:38,77697395249488,CN,PAYMENT,GBP,20637.21,3,2021-05-01 19:00:38,23526.4194,-0.115031,0
2,C12976926337644,UK,N,2021-05-23 22:15:15,77697395249488,CN,PAYMENT,USD,56691.27,22,2021-05-23 14:15:15,56691.2700,-0.012516,0
3,C12976926337644,UK,N,2021-07-07 21:36:50,77697395249488,CN,PAYMENT,CNY,343864.71,22,2021-07-07 13:36:50,48141.0594,-0.038945,0
4,C12976926337644,UK,N,2021-09-02 00:14:03,77697395249488,CN,PAYMENT,USD,57247.35,0,2021-09-01 16:14:03,57247.3500,-0.010797,0


In [4]:
trxns.outside_normal_hrs.value_counts()

0    4767
1     535
Name: outside_normal_hrs, dtype: int64

In [6]:
country_ccy_map = {
    'UK': 'GBP',
    'US': 'USD',
    'SG': 'SGD',
    'JP': 'JPY',
    'CN': 'CNY',
    'DE': 'EUR',
    'FR': 'EUR',
    'HK': 'HKD',
    'IN': 'INR',
    'BR': 'BRL'
}

In [7]:
customer_local_ccy = []
for customer_cntry, ccy in zip(trxns.customer_country, trxns.ccy):
    if ccy == country_ccy_map.get(customer_cntry):
        customer_local_ccy.append(1)
    else:
        customer_local_ccy.append(0)

In [8]:
trxns['customer_local_ccy'] = customer_local_ccy

In [9]:
no_common_ccy = []
for customer_cntry, counter_party_cntry, ccy in zip(trxns.customer_country, trxns.counterparty_country, trxns.ccy):
    if (ccy != country_ccy_map.get(counter_party_cntry)) & (ccy != country_ccy_map.get(customer_cntry)):
        no_common_ccy.append(1)
    else:
        no_common_ccy.append(0)

In [10]:
trxns['no_common_ccy'] = no_common_ccy
trxns.head()

,customer,customer_country,fraud_flag,timestamp,counterparty,counterparty_country,type,ccy,amount,hr_rounded,converted_time,converted_amount,scaled_amount,outside_normal_hrs,customer_local_ccy,no_common_ccy
0,C12976926337644,UK,N,2021-03-20 05:59:25,77697395249488,CN,OTHER,JPY,48625.25,6,2021-03-19 21:59:25,330.6517,-0.186732,0,0,1
1,C12976926337644,UK,N,2021-05-02 03:00:38,77697395249488,CN,PAYMENT,GBP,20637.21,3,2021-05-01 19:00:38,23526.4194,-0.115031,0,1,0
2,C12976926337644,UK,N,2021-05-23 22:15:15,77697395249488,CN,PAYMENT,USD,56691.27,22,2021-05-23 14:15:15,56691.2700,-0.012516,0,0,1
3,C12976926337644,UK,N,2021-07-07 21:36:50,77697395249488,CN,PAYMENT,CNY,343864.71,22,2021-07-07 13:36:50,48141.0594,-0.038945,0,0,0
4,C12976926337644,UK,N,2021-09-02 00:14:03,77697395249488,CN,PAYMENT,USD,57247.35,0,2021-09-01 16:14:03,57247.3500,-0.010797,0,0,1


In [11]:
# transform type of transaction into dummary variables
trxns = pd.get_dummies(trxns, columns=['type'], drop_first=True)
trxns.head()

,customer,customer_country,fraud_flag,timestamp,counterparty,counterparty_country,ccy,amount,hr_rounded,converted_time,...,outside_normal_hrs,customer_local_ccy,no_common_ccy,type_DIVIDEND,type_INTEREST,type_INVESTMENT,type_OTHER,type_PAYMENT,type_TRANSFER,type_TT
0,C12976926337644,UK,N,2021-03-20 05:59:25,77697395249488,CN,JPY,48625.25,6,2021-03-19 21:59:25,...,0,0,1,0,0,0,1,0,0,0
1,C12976926337644,UK,N,2021-05-02 03:00:38,77697395249488,CN,GBP,20637.21,3,2021-05-01 19:00:38,...,0,1,0,0,0,0,0,1,0,0
2,C12976926337644,UK,N,2021-05-23 22:15:15,77697395249488,CN,USD,56691.27,22,2021-05-23 14:15:15,...,0,0,1,0,0,0,0,1,0,0
3,C12976926337644,UK,N,2021-07-07 21:36:50,77697395249488,CN,CNY,343864.71,22,2021-07-07 13:36:50,...,0,0,0,0,0,0,0,1,0,0
4,C12976926337644,UK,N,2021-09-02 00:14:03,77697395249488,CN,USD,57247.35,0,2021-09-01 16:14:03,...,0,0,1,0,0,0,0,1,0,0


In [12]:
# transform type of transaction into dummary variables
trxns = pd.get_dummies(trxns, columns=['counterparty_country'], drop_first=True)
trxns.head()

,customer,customer_country,fraud_flag,timestamp,counterparty,ccy,amount,hr_rounded,converted_time,converted_amount,...,type_TT,counterparty_country_CN,counterparty_country_DE,counterparty_country_FR,counterparty_country_HK,counterparty_country_IN,counterparty_country_JP,counterparty_country_SG,counterparty_country_UK,counterparty_country_US
0,C12976926337644,UK,N,2021-03-20 05:59:25,77697395249488,JPY,48625.25,6,2021-03-19 21:59:25,330.6517,...,0,1,0,0,0,0,0,0,0,0
1,C12976926337644,UK,N,2021-05-02 03:00:38,77697395249488,GBP,20637.21,3,2021-05-01 19:00:38,23526.4194,...,0,1,0,0,0,0,0,0,0,0
2,C12976926337644,UK,N,2021-05-23 22:15:15,77697395249488,USD,56691.27,22,2021-05-23 14:15:15,56691.2700,...,0,1,0,0,0,0,0,0,0,0
3,C12976926337644,UK,N,2021-07-07 21:36:50,77697395249488,CNY,343864.71,22,2021-07-07 13:36:50,48141.0594,...,0,1,0,0,0,0,0,0,0,0
4,C12976926337644,UK,N,2021-09-02 00:14:03,77697395249488,USD,57247.35,0,2021-09-01 16:14:03,57247.3500,...,0,1,0,0,0,0,0,0,0,0


In [13]:
# finally convert the dependent variable to binary encoding
trxns['fraud_flag'] = np.where(trxns.fraud_flag=='Y', 1, 0)
trxns.head()


,customer,customer_country,fraud_flag,timestamp,counterparty,ccy,amount,hr_rounded,converted_time,converted_amount,...,type_TT,counterparty_country_CN,counterparty_country_DE,counterparty_country_FR,counterparty_country_HK,counterparty_country_IN,counterparty_country_JP,counterparty_country_SG,counterparty_country_UK,counterparty_country_US
0,C12976926337644,UK,0,2021-03-20 05:59:25,77697395249488,JPY,48625.25,6,2021-03-19 21:59:25,330.6517,...,0,1,0,0,0,0,0,0,0,0
1,C12976926337644,UK,0,2021-05-02 03:00:38,77697395249488,GBP,20637.21,3,2021-05-01 19:00:38,23526.4194,...,0,1,0,0,0,0,0,0,0,0
2,C12976926337644,UK,0,2021-05-23 22:15:15,77697395249488,USD,56691.27,22,2021-05-23 14:15:15,56691.2700,...,0,1,0,0,0,0,0,0,0,0
3,C12976926337644,UK,0,2021-07-07 21:36:50,77697395249488,CNY,343864.71,22,2021-07-07 13:36:50,48141.0594,...,0,1,0,0,0,0,0,0,0,0
4,C12976926337644,UK,0,2021-09-02 00:14:03,77697395249488,USD,57247.35,0,2021-09-01 16:14:03,57247.3500,...,0,1,0,0,0,0,0,0,0,0


In [14]:
trxns_final = trxns[
    [
        'customer', 
        'customer_country', 
        'counterparty', 
        'ccy', 
        'converted_time', 
        'scaled_amount',
        'type_DIVIDEND', 
        'type_INTEREST',
        'type_INVESTMENT', 
        'type_OTHER', 
        'type_PAYMENT', 
        'type_TRANSFER',
        'type_TT',
        'counterparty_country_CN', 
        'counterparty_country_DE',
        'counterparty_country_FR', 
        'counterparty_country_HK',
        'counterparty_country_IN', 
        'counterparty_country_JP',
        'counterparty_country_SG', 
        'counterparty_country_UK',
        'counterparty_country_US', 
        'outside_normal_hrs',
        'customer_local_ccy', 
        'no_common_ccy',
        'fraud_flag'
    ]
]

In [15]:
trxns_final.to_csv('../data/featuer_engineering_output.csv', index=False)